# SWB Prospect Theory Modeling 
Implement pyEM MAP parameter estimation method to estimate risk aversion, loss aversion, and inverse temp free parameters for every swb subject

Updated: 11/20/2024

In [3]:
import numpy as np
import pandas as pd
from glob import glob
# from numpy.linalg import vecdot
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr, norm 
import scipy.stats as stats
from scipy.optimize import least_squares, minimize
from sklearn.metrics import r2_score
import random
import os
# from statannot import add_stat_annotation
from statsmodels.stats.outliers_influence import variance_inflation_factor
import itertools
import datetime
import statsmodels.api as sm
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle
import sys

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.filterwarnings("ignore")

In [4]:
%reload_ext autoreload
%autoreload 2

In [ ]:
base_dir  = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/'
paper_dir = f'{base_dir}manuscript/'
behav_dir = f'{paper_dir}data/behav/'
fig_dir   = f'{paper_dir}figs/behav/'
save_dir  = f'{paper_dir}results/behav/'
os.makedirs(fig_dir,exist_ok=True)
os.makedirs(save_dir,exist_ok=True)

date = datetime.date.today().strftime('%m%d%Y')
print(date)


In [6]:
sys.path.append(f'{paper_dir}scripts/behav/')
from behav_utils import *
from swb_subj_behav import *
from pt_models import *
sys.path.append(f'{paper_dir}scripts/behav/pyEM-SWB-main/')
from pyEM import fitting
from pyEM import plotting
from pyEM import math


In [7]:
date = datetime.date.today().strftime('%m%d%Y')
print(date)

11212024


## Format behav data

In [9]:
subj_ids = pd.read_excel(f'{base_dir}SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID.tolist()
# subj_ids

In [10]:
# load task_dfs into master list 
raw_behav = [pd.read_csv(f'{behav_dir}single_subj/{subj_id}_task_df.csv') for subj_id in subj_ids]
all_behav,beh_drops = format_all_behav(raw_behav,return_drops=True,norm=False)
all_behav.to_csv(f'{save_dir}all_behav_pt_inputs_{date}.csv')

all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,GambleEV_t1,TrialEV_t1,CR_t1,choiceEV_t1,rpe_t1,res_type_t1,cf_t1,cpe_t1,keep_epoch,keep_epoch_t1
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,-0.475,-0.416667,NaN,-0.475,-0.475,gamble_bad,-0.30,-0.65,keep,keep
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,0.840,0.693333,NaN,0.840,0.840,gamble_good,0.40,1.28,keep,keep
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,0.200,0.133333,0.0,NaN,0.000,safe_good,-0.80,0.80,keep,keep
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,0.620,0.580000,NaN,0.620,0.620,gamble_good,0.50,0.74,keep,keep
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,-0.185,-0.123333,0.0,NaN,0.000,safe_good,-1.10,1.10,keep,keep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,-0.450,-0.300000,0.0,NaN,0.000,safe_good,-1.50,1.50,keep,keep
4046,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,-0.045,-0.030000,0.0,NaN,0.000,safe_bad,0.41,-0.41,keep,keep
4047,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,-0.190,-0.126667,0.0,NaN,0.000,safe_bad,0.42,-0.42,keep,keep
4048,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,-1.250,-1.000000,NaN,-1.250,-1.250,gamble_bad,-0.50,-2.00,keep,keep


In [11]:
# pyEM takes behav as list of dfs rather than one list - 
behav_list = [all_behav[all_behav.subj_id==subj_id].reset_index(drop=True) for subj_id in all_behav.subj_id.unique().tolist()]

## SWB EM_MAP Functions

Risk aversion, loss aversion, and inverse temp free parameters require specific bounds. Before using these parameters to estimate the negative log likelihood, you must transform them from the gaussian space back into the native model space. Here, sigmoid functions are used in the form:

native_parameter_value = upper_bound / (1 + np.exp(-gaussian_parameter_value))

The upper bound for all free parameters was determined via simulation. Upper bounds for all parameters are tested with a fixed prior before performing full estimation. 

#### Testing PT function negative log likelihood calculation with fixed prior
Risk upper bound = 2
Loss upper bound = 6
InvTemp upper bound = 10

In [12]:
#set fixed prior for function testing  
param_names = ['risk_aversion','loss_aversion','inverse_temp']
nparams = len(param_names)
prior= {}
prior['mu']    = np.abs(0.1 * np.random.randn(nparams, 1)) #random means for prior distribution in gaussian space 
prior['sigma'] = np.full((nparams, 1), 100) # prior dist std
prior['logpdf'] = lambda x: np.sum(norm.logpdf(x, prior['mu'],np.sqrt(prior['sigma']))) #calculates separate normpdfs per parameter and sums their logs


In [13]:
#testing negll function & finding maximum parameter bounds - make sure none of the negll values = 10000000

#set params to desired upper bound 
params = np.array([2,6,10]) #risk, loss, invtemp
negll_list = []

for s in range(len(subj_ids)):
    df = behav_list[s]
    negll = negll_base_pt_pyEM(params, df,prior=prior, output='npl')
    negll_list.append(negll)
    print(subj_ids[s],negll)

MS002 857.809232541126
MS003 1700.178422217988
MS009 981.1173795070584
MS011 813.3616156091296
MS015 2249.9067401601474
MS016 1330.8215182217673
MS017 516.2595499331476
MS019 2316.9250439027123
MS022 1362.8993859240272
MS024 523.7840712476566
MS025 595.2591099105432
MS026 572.6492330572561
MS027 1192.8677955683438
MS028 639.0726744607098
MS029 1531.179942761304
MS030 1255.2219572258005
MS033 1035.8548533040396
MS035 1370.0553989797163
MS041 1831.8956236808942
MS043 386.0950080968151
MS048 1062.9534639902788
MS050 546.1737487925134
DA8 194.51260835519025
DA023 839.1107414299372
DA026 1167.8046802056963
DA037 1374.5939175812036
DA039 1099.308868599731


## Parameter Estimation with MAP using pyEM 

In [14]:
#run pyEM on all subjects 

param_names = ['risk_aversion','loss_aversion','inverse_temp']
# m, inv_h, posterior, NPL, NLPrior, NLL = pyEM.fitting.EMfit(behav_list, negll_base_pt_pyEM, param_names)
output_dict = fitting.EMfit(behav_list, swb_pt_pyEM, param_names)
m, inv_h, posterior, NPL, NLPrior, NLL, convergence = output_dict.values()
#reorganize and rename outputs!

2961.572 (000), 2327.459 (001), 2312.245 (002), 2308.099 (003), 2306.633 (004), 2306.052 (005), 2305.804 (006), 2305.694 (007), 2305.620 (008), 2305.616 (009), 2305.574 (010), 2305.550 (012),  -- CONVERGED!!!!!


### Extract parameter estimates 

In [170]:
#transform gaussian params into native model space (using sigmoid function)

param_names = ['risk_aversion','loss_aversion','inverse_temp']
nparams = len(param_names)

#extract estimated parameter values in gaussian space  
est_params = m.T.copy() #rows = subj, cols = params


for subj_idx in range(len(subj_ids)): #iterate through each subject

    for param_idx, param_name in enumerate(param_names): #iterate through each free parameter 
        if 'risk_aversion' == param_name:
            est_params[subj_idx, param_idx] = norm2riskaversion(m[param_idx, subj_idx]) #replace gaussian value with transformed value in param matrix 
        elif 'loss_aversion' == param_name:
            est_params[subj_idx, param_idx] = norm2lossaversion(m[param_idx, subj_idx]) #replace gaussian value with transformed value in param matrix 
        elif 'inverse_temp' == param_name: 
            est_params[subj_idx, param_idx] = norm2invtmp(m[param_idx, subj_idx]) #replace gaussian value with transformed value in param matrix 

list(zip(subj_ids,est_params)) #each row is the 3 param estimates for each subj 
# can also make this a dataframe next time 

[('MS002', array([0.81907367, 2.37237889, 2.24950998])),
 ('MS003', array([0.75959037, 0.10841149, 5.16127126])),
 ('MS009', array([1.13755074, 1.4861448 , 1.53000205])),
 ('MS011', array([1.29848375, 1.14729727, 2.05632935])),
 ('MS015', array([1.08376638, 0.07804721, 2.36562243])),
 ('MS016', array([0.63980355, 0.97994856, 6.01634943])),
 ('MS017', array([1.3072073 , 1.00217865, 6.03129817])),
 ('MS019', array([1.10733439, 0.04001488, 6.05516146])),
 ('MS022', array([1.32108743, 0.22755131, 4.5521941 ])),
 ('MS024', array([0.67843141, 3.95065394, 2.09100956])),
 ('MS025', array([0.82676446, 4.20454112, 1.64358111])),
 ('MS026', array([1.31816733, 1.5951847 , 2.43293795])),
 ('MS027', array([0.66546819, 0.65564952, 5.13421896])),
 ('MS028', array([1.55293246, 0.65492881, 7.78321653])),
 ('MS029', array([0.58948182, 0.62031575, 2.09216392])),
 ('MS030', array([0.75799955, 1.135369  , 4.99472728])),
 ('MS033', array([1.49955899, 0.35837352, 5.91264193])),
 ('MS035', array([1.10760677, 0

In [171]:
est_params

array([[0.81907367, 2.37237889, 2.24950998],
       [0.75959037, 0.10841149, 5.16127126],
       [1.13755074, 1.4861448 , 1.53000205],
       [1.29848375, 1.14729727, 2.05632935],
       [1.08376638, 0.07804721, 2.36562243],
       [0.63980355, 0.97994856, 6.01634943],
       [1.3072073 , 1.00217865, 6.03129817],
       [1.10733439, 0.04001488, 6.05516146],
       [1.32108743, 0.22755131, 4.5521941 ],
       [0.67843141, 3.95065394, 2.09100956],
       [0.82676446, 4.20454112, 1.64358111],
       [1.31816733, 1.5951847 , 2.43293795],
       [0.66546819, 0.65564952, 5.13421896],
       [1.55293246, 0.65492881, 7.78321653],
       [0.58948182, 0.62031575, 2.09216392],
       [0.75799955, 1.135369  , 4.99472728],
       [1.49955899, 0.35837352, 5.91264193],
       [1.10760677, 0.64603689, 1.93377233],
       [1.51704585, 0.08572707, 4.00442395],
       [1.21714511, 1.16127844, 8.89642751],
       [0.89011602, 1.61769065, 1.64582775],
       [1.20734707, 0.88854158, 8.70764767],
       [1.

In [172]:
# compute mean, std of posterior distributions & covariance matrix from inv hessians (just non-diag elements of inv hessian for all param combos)
posterior_mu, posterior_std, _, covariance_mat = math.compGauss_ms(m,inv_h,2) #posterior mu/std is same here as in EMfit output 

covariance_mat #covariance between free parameters 

array([[ 0.48875686, -0.31953792,  0.14085668],
       [-0.31953792,  2.30727606, -0.49823605],
       [ 0.14085668, -0.49823605,  1.32130256]])

In [173]:
# log model evidence calculation
Laplace_approx, lme, goodHessian = math.calc_LME(inv_h, NPL)

Good Hessians: 27 out of 27


In [174]:
Laplace_approx

array([ -93.95559436,  -91.05799916, -101.91486559,  -96.73288854,
        -95.07729055,  -93.01669864,  -71.47742213,  -83.37965714,
        -84.18325893,  -67.51630064,  -86.23211003,  -87.9134572 ,
       -100.43931247,  -60.04550665, -104.34548247,  -95.26067154,
        -77.44366742, -101.07212579,  -88.8730928 ,  -47.14390586,
       -103.68262912,  -54.7707543 ,  -60.8570357 ,  -95.66897496,
        -94.89783868,  -88.15474127,  -95.50428658])

In [175]:
lme

-2330.505079113208

In [176]:
# integrated BIC calculation 
# bic_int = math.calc_BICint(behav_list, param_names, posterior['mu'], posterior['sigma'], negll_base_pt_pyEM)

# # Define settings
  
# mu = posterior['mu']
# sigma = posterior['sigma']
# nsamples = 2000
# nll_output='negll'
# func_output='all'

# npar = len(param_names)      
# total_trials = behav_list[0].size

# # Convert to std dev
# sigmasqrt = np.sqrt(posterior['sigma'])

# # Initialize
# iLog = np.empty(len(behav_list))

# # Start computing
# for isub, beh in enumerate(behav_list):        
#     # Sample parameters from the Gaussian distribution
#     Gsamples = norm.rvs(loc=np.tile(mu[:, np.newaxis], (1, nsamples)), scale=np.tile(sigmasqrt[:, np.newaxis], (1, nsamples)))

#     # Compute negative log likelihood for each sample
#     subnll = Parallel(n_jobs=-1)(delayed(lambda k: negll_base_pt_pyEM(*([Gsamples[:, k]] + beh), output=func_output)[nll_output])(k) for k in range(nsamples))

#     # Compute integrated log likelihood
#     # iLog[isub] = np.log(np.sum(np.exp(-np.array(subnll))) / nsamples)

# # Compute BICint
# # bicint = -2 * np.sum(iLog) + npar * np.log(total_trials)

In [177]:
#combine results into dictionary to save 
swb_pt_MAP_dict = {}

swb_pt_MAP_dict['subj_ids'] = subj_ids
swb_pt_MAP_dict['gauss_params'] = m
swb_pt_MAP_dict['params'] = est_params
swb_pt_MAP_dict['param_names'] = param_names
swb_pt_MAP_dict['inverse_hess'] = inv_h
swb_pt_MAP_dict['gauss_posterior_mu'] = posterior['mu']
swb_pt_MAP_dict['gauss_posterior_std'] = posterior['sigma']
swb_pt_MAP_dict['gauss_covariance_mat'] = covariance_mat
swb_pt_MAP_dict['npl'] = NPL #negative joint posterior likelihood (current convergence criteria)
swb_pt_MAP_dict['NLPrior'] = NLPrior #negative log prior estimate
swb_pt_MAP_dict['nll'] =  NPL - NLPrior #negative log likelihood (neg joint posterior ll - neg log prior)
swb_pt_MAP_dict['bic'] =  np.log(150)*nparams + 2*swb_pt_MAP_dict['nll'] 
# swb_pt_MAP_dict['aic'] =  2*nparams + 2*swb_pt_MAP_dict['nll'] 
swb_pt_MAP_dict['lme'] = [] #not calculated yet - convergence type = npl instead 

# swb_pt_MAP_dict

In [178]:
swb_pt_MAP_dict['bic'][0] # ask shawn why this is not just one number per subj?

192.7837851253728

# Make output dfs

In [179]:
subj_pt_dir  = f'{save_dir}subj_pt_data/'
os.makedirs(subj_pt_dir,exist_ok=True)

In [180]:
# Use parameter estimations to fit pt model to subjects and save data dicts --- remember to use gaussian form of vars!  est_params[subj_idx,:]
swb_pt_results = []
pt_all_behav   = []

for subj_idx,subj_id in enumerate(subj_ids):
    params = m[:,subj_idx] #make sure these are the gaussian parameter values 
    df = behav_list[subj_idx]

    subj_df,subj_res = swb_pt_pyEM(params, df, prior=None, output='all')
    subj_df['risk'] = subj_res['params'][0]
    subj_df['loss'] = subj_res['params'][1]
    subj_df['temp'] = subj_res['params'][2]
    subj_df.to_csv(f'{subj_pt_dir}{subj_id}_pt_df.csv')

    pt_all_behav.append(subj_df)
    swb_pt_results.append(pd.DataFrame({'subj_id':subj_df.subj_id.unique()[0],
                                        'bdi':subj_df.bdi.unique()[0],
                                        'risk':subj_res['params'][0],
                                        'loss':subj_res['params'][1],
                                        'temp':subj_res['params'][2],
                                        'negll':subj_res['negll'],
                                        'bic':subj_res['bic']},index=[0]))



In [181]:
pt_all_behav = pd.concat(pt_all_behav).reset_index(drop=True)
pt_all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,util_LowBet,util_SafeBet,util_Gamble,P_Gamble,P_Safe,Choice_Prob,Choice_Util,risk,loss,temp
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,-1.282501,0.000000,-0.966991,0.101995,0.898005,0.898005,0.000000,0.819074,2.372379,2.249510
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,-1.137386,-0.884928,-1.137386,0.361720,0.638280,0.361720,-1.137386,0.819074,2.372379,2.249510
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,-0.000000,0.472126,0.764742,0.658866,0.341134,0.658866,0.764742,0.819074,2.372379,2.249510
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,-0.988047,0.000000,-0.407516,0.285628,0.714372,0.714372,0.000000,0.819074,2.372379,2.249510
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,-0.000000,0.566806,0.596334,0.516600,0.483400,0.516600,0.596334,0.819074,2.372379,2.249510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,-0.000000,0.337042,0.925529,0.790182,0.209818,0.790182,0.925529,1.186903,0.864265,2.253272
4046,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,-0.699230,0.000000,-0.426548,0.276652,0.723348,0.723348,0.000000,1.186903,0.864265,2.253272
4047,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,-0.189812,0.000000,-0.016279,0.490831,0.509169,0.509169,0.000000,1.186903,0.864265,2.253272
4048,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,-0.331585,0.000000,-0.153017,0.414647,0.585353,0.585353,0.000000,1.186903,0.864265,2.253272


In [182]:
pt_all_behav.to_csv(f'{save_dir}pt_all_behav.csv')
pt_all_behav.to_csv(f'{save_dir}pt_all_behav_{date}.csv')


In [183]:
swb_pt_results = pd.concat(swb_pt_results).reset_index(drop=True)
swb_pt_results

,subj_id,bdi,risk,loss,temp,negll,bic
0,MS002,14,0.819074,2.372379,2.249510,88.875909,192.783724
1,MS003,8,0.759590,0.108411,5.161271,86.620882,188.233400
2,MS009,16,1.137551,1.486145,1.530002,97.829936,210.691778
3,MS011,13,1.298484,1.147297,2.056329,92.428541,199.868920
4,MS015,26,1.083766,0.078047,2.365622,91.280910,197.408100
5,MS016,10,0.639804,0.979949,6.016349,87.169352,189.227238
6,MS017,26,1.307207,1.002179,6.031298,66.970518,148.972941
7,MS019,12,1.107334,0.040015,6.055161,78.448397,171.908634
8,MS022,10,1.321087,0.227551,4.552194,80.722204,176.374610
9,MS024,16,0.678431,3.950654,2.091010,61.492015,137.420583


In [184]:
swb_pt_results.to_csv(f'{save_dir}swb_pt_results.csv')
swb_pt_results.to_csv(f'{save_dir}swb_pt_results_{date}.csv')


In [185]:
pt_all_behav

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,util_LowBet,util_SafeBet,util_Gamble,P_Gamble,P_Safe,Choice_Prob,Choice_Util,risk,loss,temp
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,-1.282501,0.000000,-0.966991,0.101995,0.898005,0.898005,0.000000,0.819074,2.372379,2.249510
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,-1.137386,-0.884928,-1.137386,0.361720,0.638280,0.361720,-1.137386,0.819074,2.372379,2.249510
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,-0.000000,0.472126,0.764742,0.658866,0.341134,0.658866,0.764742,0.819074,2.372379,2.249510
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,-0.988047,0.000000,-0.407516,0.285628,0.714372,0.714372,0.000000,0.819074,2.372379,2.249510
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,-0.000000,0.566806,0.596334,0.516600,0.483400,0.516600,0.596334,0.819074,2.372379,2.249510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,-0.000000,0.337042,0.925529,0.790182,0.209818,0.790182,0.925529,1.186903,0.864265,2.253272
4046,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,-0.699230,0.000000,-0.426548,0.276652,0.723348,0.723348,0.000000,1.186903,0.864265,2.253272
4047,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,-0.189812,0.000000,-0.016279,0.490831,0.509169,0.509169,0.000000,1.186903,0.864265,2.253272
4048,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,-0.331585,0.000000,-0.153017,0.414647,0.585353,0.585353,0.000000,1.186903,0.864265,2.253272


In [190]:
raw_behav = [pd.read_csv(f'{behav_dir}single_subj/{subj_id}_task_df.csv') for subj_id in subj_ids]

pt_all_behav_master = format_all_behav(raw_behav, return_drops=False, drop_bads=False, drop_bads_t1=False,
                     norm=True,norm_type='zscore',pt_dir=None)
pt_all_behav_master.to_csv(f'{save_dir}pt_all_behav_master.csv')

In [191]:
pt_all_behav_master

,subj_id,bdi,bdi_thresh,Round,TrialNum,RT,TrialOnset,ChoiceOnset,DecisionOnset,FeedbackOnset,...,HighBet_t1_raw,Profit_t1_raw,TotalProfit_t1_raw,GambleEV_t1_raw,TrialEV_t1_raw,CR_t1_raw,choiceEV_t1_raw,rpe_t1_raw,cf_t1_raw,cpe_t1_raw
0,MS002,14,low,1,25.0,2.059852,513.380590,513.390239,515.450091,515.457173,...,0.00,-0.95,9.05,-0.475,-0.416667,NaN,-0.475,-0.475,-0.30,-0.65
1,MS002,14,low,2,117.0,1.954564,522.640856,522.641563,524.596127,526.627092,...,1.68,1.68,10.73,0.840,0.693333,NaN,0.840,0.840,0.40,1.28
2,MS002,14,low,3,79.0,1.583462,531.174799,531.175599,532.759061,534.780269,...,1.20,0.00,10.73,0.200,0.133333,0.0,NaN,0.000,-0.80,0.80
3,MS002,14,low,4,42.0,2.491611,545.592613,545.593355,548.084966,548.092333,...,1.24,1.24,11.97,0.620,0.580000,NaN,0.620,0.620,0.50,0.74
4,MS002,14,low,5,85.0,1.768936,555.337336,555.345720,557.114656,559.135069,...,0.73,0.00,11.97,-0.185,-0.123333,0.0,NaN,0.000,-1.10,1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4045,DA039,22,high,146,79.0,1.079701,2259.827656,2259.828749,2260.908450,2262.926195,...,0.60,0.00,18.95,-0.450,-0.300000,0.0,NaN,0.000,-1.50,1.50
4046,DA039,22,high,147,30.0,1.837272,2267.502359,2267.534059,2269.371331,2269.377701,...,0.41,0.00,18.95,-0.045,-0.030000,0.0,NaN,0.000,0.41,-0.41
4047,DA039,22,high,148,13.0,4.030006,2282.349445,2282.350662,2286.380667,2286.389886,...,0.42,0.00,18.95,-0.190,-0.126667,0.0,NaN,0.000,0.42,-0.42
4048,DA039,22,high,149,18.0,3.167144,2293.040983,2293.042042,2296.209186,2296.218136,...,0.00,-2.50,16.45,-1.250,-1.000000,NaN,-1.250,-1.250,-0.50,-2.00


In [ ]:
#### Model Evaluation - Laplace Approximation (Bayes Factor similar to BIC that accounts for prior distribution, covariance from hessians, and paramater number)

# from mfit_optimize_hierarchical.m from Sam Gershman
# Also reference Daw 2009 (Equation 17) for Laplace approximation

# goodHessian = np.zeros(len(subj_ids))
# modelID = 'negll_base_pt_pyEM'
# modout = {}
# modout[modelID] = {}
# modout[modelID]['fit'] = {}
# for subj_idx in range(len(subj_ids)):
#     try:
#         det_inv_hessian = np.linalg.det(inv_h[:, :, subj_idx])
#         hHere = np.linalg.slogdet(inv_h[:, :, subj_idx])[1]
#         L = -NPL - 0.5 * np.log(1 / det_inv_hessian) + (nparams / 2) * np.log(2 * np.pi)
#         goodHessian[subj_idx] = 1
#     except:
#         print('Hessian is not positive definite')
#         try:
#             hHere = np.linalg.slogdet(inv_h[:,:,subj_idx])[1]
#             L = np.nan
#             goodHessian[subj_idx] = 0
#         except:
#             print('could not calculate')
#             goodHessian[subj_idx] = -1
#             L = np.nan
#     modout[modelID]['fit']['lme'] = L
#     modout[modelID]['fit']['goodHessian'] = goodHessian

#np.sum(modout[modelID]['fit']['goodHessian']==1)


# goodHessian = np.zeros(len(subj_ids))
# modelID = 'negll_base_pt_pyEM'
# modout = {}
# modout[modelID] = {}
# modout[modelID]['fit'] = {}
# for subj_idx in range(len(subj_ids)):
#     try:
#         det_inv_hessian = np.linalg.det(inv_h[:, :, subj_idx])
#         hHere = np.linalg.slogdet(inv_h[:, :, subj_idx])[1]
#         L = -NPL - 0.5 * np.log(1 / det_inv_hessian) + (nparams / 2) * np.log(2 * np.pi)
#         goodHessian[subj_idx] = 1
#     except:
#         print('Hessian is not positive definite')
#         try:
#             hHere = np.linalg.slogdet(inv_h[:,:,subj_idx])[1]
#             L = np.nan
#             goodHessian[subj_idx] = 0
#         except:
#             print('could not calculate')
#             goodHessian[subj_idx] = -1
#             L = np.nan
#     modout[modelID]['fit']['lme'] = L
#     modout[modelID]['fit']['goodHessian'] = goodHessian


# Make sure you know if BIC is positive or negative! and replace lme with
# bic if covariance is negative.
# Error check that BICs are in a similar range

In [ ]:
# ############### code from shawn bayesian model selection    "integrated BIC" 
# #here, the mean and variance from a hierarchical model fit are use to generate 2000 samples of paramters, 
    # but you can still use the mean and variance from the individual fits in your data. 
    # fit.beh is just the behavioral data for all subjects. fit.objfunc is the function for outputing the negll

# #https://www.nature.com/articles/s41467-020-17343-w

# Nsample     = 2000

# # % info for normpdf, and flip if it is the wrong orientation
# mu          = mroot.(modelID).gauss.mu;            if size(mu,2)>size(mu,1),                 mu = mu'; end
# sigmasqrt   = sqrt(mroot.(modelID).gauss.sigma);   if size(sigmasqrt,2)>size(sigmasqrt,1),   sigmasqrt = sigmasqrt'; end

# # % collect integrated nll
# iLog        = nan(numel(fit.beh),1); 

# # % 1)get integrated nll by sampling nll from group gaussian
# fprintf([modelID ' - BICint: ']);
# for is = 1:numel(fit.beh)
   
#    subnll = nan(1,Nsample);
#    Gsamples    = normrnd(repmat(mu,1,Nsample),repmat(sigmasqrt,1,Nsample));  % samples from gaussian distribution found during EM; draw anew for each subject

# #    % for each subject, get NLL for input params from gaussian
#    for k=1:Nsample
#       subnll(k)  = fit.objfunc(fit.beh{is},Gsamples(:,k)); 
#    end
#    fprintf([ num2str(is) ',']);
#    iLog(is) = log(sum(exp(-subnll))/Nsample);
# end

# % 2) Compute BICint
# bicint  = -2*sum(iLog)   + fit.npar*log(sum(fit.ntrials)); 